In [1]:
import requests
import time
import pandas as pd
import numpy as np
import psycopg2
import sqlalchemy


In [3]:
host = "34.70.251.126"
port = 5432
user = "admin"
password = "root"
database= "umweltmonitoring"

### Messwerte der einzelnen Stationen in ein Dataframe speichern

In [7]:
#Messstationen definieren
boxes_dic = {
    "Mersch Luxembourg" : "5ae4a726223bd80019a367a6",
    "Ettlingen Deutschland" : "5dde9523ba7944001da4150e",
    "Stuttgart Deutschland" : "5c08379b1c28f9001a3f580c",
    "München Deutschland" : "5b4d11485dc1ec001b5452c7",
    "Berlin Deutschland" : "65e8d93acbf5700007f920ca",
    "Hamburg Deutschland" : "590e0b0a51d3460011c725c4",
    "Dortmund Deutschland" : "605f498077a88b001bba3dc0",
    "Jena Deutschland" : "64722d1c9be0580007f776d9"
}

In [4]:
# Erstelle ein leeres DataFrame mit Spaltennamen
df_messwerte = pd.DataFrame(columns=["time", "temperature","luftfeuchtigkeit", "pm10", "pm2_5", "senseid_fk"])

In [5]:

# Assuming df_messwerte is already defined as a DataFrame

new_data = []  # A list to collect new data
for box_id, box_value in boxes_dic.items():
    
    try:
        response = requests.get(f'https://api.opensensemap.org/boxes/{box_value}?format=json')
        box = response.json() 
    except:
        print("FUCK YOU SENSEBOX NO WORKING TODAY" + box_value)
    temperature_value = None
    pm10_value = None
    pm25_value = None
    rel_Luftfeuchte_value = None

    # Traverse through the 'sensors' list
    for sensor in box.get('sensors', []):
        # Check if the sensor is for Temperature
        if sensor.get('title') == 'Temperatur':
            # Access the temperature value
            temperature_value = sensor.get('lastMeasurement', {}).get('value')
        
        if sensor.get('title') == 'PM10':
            # Access the PM10 value
            pm10_value = sensor.get('lastMeasurement', {}).get('value')

        if sensor.get('title') == 'PM2.5':
            # Access the PM2.5 value
            pm25_value = sensor.get('lastMeasurement', {}).get('value')

        if sensor.get('title') == 'rel. Luftfeuchte':
            # Access the relative humidity value
            rel_Luftfeuchte_value = sensor.get('lastMeasurement', {}).get('value')
            
            break  # Exit the loop once the temperature is found
        
    # Append new data to the list
    new_data.append({"temperature": temperature_value, "time": time.strftime("%Y-%m-%d %H:%M:%S"),
                     "luftfeuchtigkeit": rel_Luftfeuchte_value, "pm10": pm10_value, "pm2_5": pm25_value,
                     "senseid_fk": box_value})
    
    # Convert the new data list to a DataFrame
new_df = pd.DataFrame(new_data)
    
    # Append the new DataFrame to the existing DataFrame
df_messwerte = pd.concat([df_messwerte, new_df], ignore_index=True)
df_messwerte = df_messwerte.fillna(0)

# Convert specific columns to float64, handling non-numeric values
columns_to_convert = ["temperature", "luftfeuchtigkeit", "pm10", "pm2_5"]
for column in columns_to_convert:
    df_messwerte[column] = pd.to_numeric(df_messwerte[column], errors='coerce')

df_messwerte = df_messwerte.round(2)


In [6]:
df_messwerte

,time,temperature,luftfeuchtigkeit,pm10,pm2_5,senseid_fk
0,2024-05-15 18:12:53,18.50,99.90,4.20,3.00,5ae4a726223bd80019a367a6
1,2024-05-15 18:12:54,18.32,73.25,7.80,7.10,5dde9523ba7944001da4150e
2,2024-05-15 18:12:54,23.30,55.60,2.83,2.63,5c08379b1c28f9001a3f580c
3,2024-05-15 18:12:54,17.24,69.25,10.77,3.38,5b4d11485dc1ec001b5452c7
4,2024-05-15 18:12:54,26.50,30.20,0.00,0.00,65e8d93acbf5700007f920ca
5,2024-05-15 18:12:54,28.90,37.00,4.87,1.50,590e0b0a51d3460011c725c4
6,2024-05-15 18:12:54,22.61,0.00,0.00,0.00,605f498077a88b001bba3dc0
7,2024-05-15 18:12:54,24.82,27.19,3.28,1.50,64722d1c9be0580007f776d9


In [7]:
df_messwerte["time"] = pd.to_datetime(df_messwerte["time"])

In [8]:
df_messwerte.dtypes

time                datetime64[ns]
temperature                float64
luftfeuchtigkeit           float64
pm10                       float64
pm2_5                      float64
senseid_fk                  object
dtype: object

In [9]:
import sqlalchemy

# Define your PostgreSQL connection string
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Create a SQLAlchemy engine
engine = sqlalchemy.create_engine(connection_string)

try:
    # Connect to the database using psycopg2
    connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
    print("Verbindung zur Datenbank erfolgreich hergestellt.")

    # Now you can proceed with inserting the DataFrame into the PostgreSQL database
    df_messwerte.to_sql("value", con=engine, if_exists="append", index=False)
    print("Daten erfolgreich in die Datenbank eingefügt.")
    
except Exception as e:
    print(f"Fehler beim Herstellen der Verbindung zur Datenbank oder Einfügen der Daten: {e}")


Verbindung zur Datenbank erfolgreich hergestellt.
Daten erfolgreich in die Datenbank eingefügt.


### Zweites DataFrame für Daten der Messstationen erstellen

In [10]:
# Erstelle ein leeres DataFrame mit Spaltennamen
df_stations = pd.DataFrame(columns=["Ort","Land","Lat", "Lon", "Hoehe", "senseID"])

In [11]:
new_data = []  # Eine leere Liste, um neue Daten zu sammeln
for box_id, box_value in boxes_dic.items():
    try:
        response = requests.get(f'https://api.opensensemap.org/boxes/{box_value}?format=json')
        box = response.json() 
    except:
        print("FUCK YOU SENSEBOX NO WORKING TODAY" + box_value)
    temperature_value = None
    lat = None
    lon = None
    hoehe = None
    pm10_value = None
    pm25_value = None
    rel_Luftfeuchte_value = None
    ort_Land = box_id.split()

    # Extrahiere Koordinaten
    current_location = box.get('currentLocation', {})
    if current_location:
        coordinates = current_location.get('coordinates')
        if coordinates:
            lon = coordinates[0]
            lat = coordinates[1]
            height = coordinates[2] if len(coordinates) >= 3 else None

        
    # Füge neue Daten zur Liste hinzu
    new_data.append({"Ort": ort_Land[0], "Land":ort_Land[1], "Lat": lat,"Lon":lon,"Hoehe":height, "senseID": box_value})
    
    # Konvertiere die neue Datenliste in ein DataFrame
new_df = pd.DataFrame(new_data)
    
    # Hänge das neue DataFrame an das bestehende DataFrame an
df_stations = pd.concat([df_stations, new_df], ignore_index=True)

FUCK YOU SENSEBOX NO WORKING TODAY590e0b0a51d3460011c725c4


In [12]:
df_stations

,Ort,Land,Lat,Lon,Hoehe,senseID
0,Mersch,Luxembourg,49.754663,6.105682,250.0,5ae4a726223bd80019a367a6
1,Ettlingen,Deutschland,48.926209,8.374409,4.0,5dde9523ba7944001da4150e
2,Stuttgart,Deutschland,48.779610,9.213723,280.0,5c08379b1c28f9001a3f580c
3,München,Deutschland,48.146546,11.576033,20.0,5b4d11485dc1ec001b5452c7
4,Berlin,Deutschland,52.511270,13.397356,45.0,65e8d93acbf5700007f920ca
5,Hamburg,Deutschland,52.511270,13.397356,45.0,590e0b0a51d3460011c725c4
6,Dortmund,Deutschland,51.497350,7.494606,NaN,605f498077a88b001bba3dc0
7,Jena,Deutschland,50.934561,11.589701,146.0,64722d1c9be0580007f776d9


### Daten von der DB lesen

In [44]:
try:
    # Connect to the database using psycopg2
    connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
    print("Verbindung zur Datenbank erfolgreich hergestellt.")

    # SQL-Abfrage ausführen
    query = "SELECT * FROM value"  
    cursor = connection.cursor()
    cursor.execute(query)
    results = cursor.fetchall()

    # DataFrame erstellen
    df = pd.DataFrame(results, columns=[desc[0] for desc in cursor.description])
    df = df.drop(columns=["index"])
    # Verbindung schließen
    connection.close()
    print("Daten erfolgreich geladen.")
    
except Exception as e:
    print(f"Fehler beim Herstellen der Verbindung zur Datenbank oder Einfügen der Daten: {e}")


Verbindung zur Datenbank erfolgreich hergestellt.
Daten erfolgreich geladen.


In [49]:
df

,time,temperature,luftfeuchtigkeit,pm10,pm2_5,senseid_fk
0,2024-05-15 16:32:38,18.30,99.90,3.40,3.10,5ae4a726223bd80019a367a6
1,2024-05-15 16:32:39,17.90,75.63,9.45,8.20,5dde9523ba7944001da4150e
2,2024-05-15 16:32:40,22.90,55.90,3.23,2.93,5c08379b1c28f9001a3f580c
3,2024-05-15 16:32:40,17.24,69.30,10.77,3.38,5b4d11485dc1ec001b5452c7
4,2024-05-15 16:32:41,26.30,30.70,0.00,0.00,65e8d93acbf5700007f920ca
...,...,...,...,...,...,...
1083,2024-05-16 15:20:03,17.46,67.98,10.77,3.38,5b4d11485dc1ec001b5452c7
1084,2024-05-16 15:20:04,26.50,29.60,0.00,0.00,65e8d93acbf5700007f920ca
1085,2024-05-16 15:20:05,30.60,51.40,5.13,2.40,590e0b0a51d3460011c725c4
1086,2024-05-16 15:20:05,18.39,0.00,0.00,0.00,605f498077a88b001bba3dc0


### plotly / dash (aus Übung)

In [48]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import requests
from datetime import datetime

In [ ]:

# Initialize Dash app
app = dash.Dash(__name__)

# Create layout
app.layout = html.Div([
    # Graph component
    dcc.Graph(id='live-graph'),

    # Triggers periodically
    dcc.Interval(
        id='interval-component',
        interval=10_000,
        n_intervals=0
    )
])

# Initialize empty dataframe
df = pd.DataFrame(columns=['timestamp', 'value'])


# Define callback to update graph
@app.callback(
    Output('live-graph', 'figure'),
    [Input('interval-component', 'n_intervals')]
)
def update_graph(n):  # increment from intervals

    # Request API
    box_id = "616dd3698bd2c9001cc6d293"
    url = f"https://api.opensensemap.org/boxes/{box_id}?format=json"
    data = requests.get(url).json()
    value = data['sensors'][2]['lastMeasurement']['value']
    timestamp = datetime.now()

    global df
    df = pd.concat([df, pd.DataFrame([{'timestamp': timestamp, 'value': float(value)}])], ignore_index=True)

    # Plot data
    trace = go.Scatter(x=df['timestamp'], y=df['value'], mode='lines+markers')
    layout = go.Layout(title='Real-time Sensor Data', xaxis=dict(title='Timestamp'), yaxis=dict(title='Value'))
    fig = go.Figure(data=[trace], layout=layout)
    return fig


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

### Random Forest Modell trainieren

In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor  # Für Regression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


In [ ]:
# Beispiel (Anpassen an Ihre Daten):
y = df["temperature"]              # Zielvariable
X = df.drop(["time","temperature"], axis=1)  # Features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definiere Spalten-Transformer, um One-Hot-Encoding für die kategoriale Spalte durchzuführen
# Annahme: category_column_index ist der Index der kategorialen Spalte in X
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ['senseid_fk'])], remainder='passthrough')

# Führe die Spalten-Transformation durch
X_encoded = ct.fit_transform(X)

# Aufteilung in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Für Regression:
model = RandomForestRegressor(n_estimators=100, random_state=42) 
model.fit(X_train, y_train)


In [32]:
y_pred = model.predict(X_test)
import numpy as np
# Für Regression:
from sklearn.metrics import mean_squared_error, r2_score
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred)}")
print(f"R-squared: {r2_score(y_test, y_pred)}")
r2 = mean_squared_error(y_test, y_pred)
print(f"RMSE: {np.sqrt(r2)}")



Mean Squared Error: 3.353794879188194
R-squared: 0.8738075065796083
RMSE: 1.831336910343969
